<a href="https://colab.research.google.com/github/Prajith1509/Prajith/blob/main/PCBmodel.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# COMPLETE PCB DEFECT DETECTION SYSTEM
# ============================================================================

# 1. MODEL TRAINING CODE
# ============================================================================

import os
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix
import cv2
import matplotlib.pyplot as plt
from PIL import Image
import pickle
import json

# Set random seeds for reproducibility
np.random.seed(42)
tf.random.set_seed(42)

class PCBDatasetGenerator:
    """Generate synthetic PCB dataset for training"""

    def __init__(self, num_samples=2000, img_size=(224, 224)):
        self.num_samples = num_samples
        self.img_size = img_size

    def create_good_pcb(self):
        """Create a synthetic good PCB image"""
        img = np.zeros((*self.img_size, 3), dtype=np.uint8)
        img.fill(34)  # Dark background

        # Add copper traces (good patterns)
        for _ in range(np.random.randint(5, 12)):
            # Horizontal traces
            y = np.random.randint(20, self.img_size[0] - 20)
            thickness = np.random.randint(3, 8)
            cv2.line(img, (10, y), (self.img_size[1] - 10, y), (139, 69, 19), thickness)

            # Vertical traces
            x = np.random.randint(20, self.img_size[1] - 20)
            cv2.line(img, (x, 10), (x, self.img_size[0] - 10), (139, 69, 19), thickness)

        # Add components (rectangles)
        for _ in range(np.random.randint(3, 8)):
            x1 = np.random.randint(30, self.img_size[1] - 60)
            y1 = np.random.randint(30, self.img_size[0] - 60)
            x2 = x1 + np.random.randint(20, 40)
            y2 = y1 + np.random.randint(15, 30)
            cv2.rectangle(img, (x1, y1), (x2, y2), (80, 80, 80), -1)

        # Add via holes
        for _ in range(np.random.randint(8, 15)):
            center = (np.random.randint(30, self.img_size[1] - 30),
                     np.random.randint(30, self.img_size[0] - 30))
            cv2.circle(img, center, np.random.randint(2, 5), (200, 200, 200), -1)

        return img

    def create_defective_pcb(self):
        """Create a synthetic defective PCB image"""
        img = self.create_good_pcb()

        # Add random defects
        defect_type = np.random.choice(['spurious_copper', 'missing_hole', 'open_circuit', 'short'])

        if defect_type == 'spurious_copper':
            # Add unwanted copper patches
            for _ in range(np.random.randint(1, 4)):
                center = (np.random.randint(50, self.img_size[1] - 50),
                         np.random.randint(50, self.img_size[0] - 50))
                cv2.circle(img, center, np.random.randint(8, 20), (139, 69, 19), -1)

        elif defect_type == 'missing_hole':
            # Cover some holes with copper
            for _ in range(np.random.randint(2, 5)):
                center = (np.random.randint(30, self.img_size[1] - 30),
                         np.random.randint(30, self.img_size[0] - 30))
                cv2.circle(img, center, 8, (139, 69, 19), -1)

        elif defect_type == 'open_circuit':
            # Break some traces
            for _ in range(np.random.randint(1, 3)):
                x = np.random.randint(50, self.img_size[1] - 50)
                y = np.random.randint(50, self.img_size[0] - 50)
                cv2.rectangle(img, (x-5, y-5), (x+5, y+5), (34, 34, 34), -1)

        elif defect_type == 'short':
            # Add unwanted connections
            for _ in range(np.random.randint(1, 3)):
                pt1 = (np.random.randint(30, self.img_size[1] - 30),
                       np.random.randint(30, self.img_size[0] - 30))
                pt2 = (pt1[0] + np.random.randint(-30, 30),
                       pt1[1] + np.random.randint(-30, 30))
                cv2.line(img, pt1, pt2, (139, 69, 19), np.random.randint(2, 6))

        return img

    def generate_dataset(self):
        """Generate complete dataset"""
        images = []
        labels = []

        print("Generating synthetic PCB dataset...")

        # Generate good PCBs
        for i in range(self.num_samples // 2):
            if i % 100 == 0:
                print(f"Generated {i} good PCBs...")
            img = self.create_good_pcb()
            images.append(img)
            labels.append(0)  # 0 = Good

        # Generate defective PCBs
        for i in range(self.num_samples // 2):
            if i % 100 == 0:
                print(f"Generated {i} defective PCBs...")
            img = self.create_defective_pcb()
            images.append(img)
            labels.append(1)  # 1 = Defective

        return np.array(images), np.array(labels)

class PCBCNNModel:
    """CNN Model for PCB Classification"""

    def __init__(self, input_shape=(224, 224, 3)):
        self.input_shape = input_shape
        self.model = None

    def build_model(self):
        """Build CNN architecture"""
        self.model = models.Sequential([
            # First Convolutional Block
            layers.Conv2D(32, (3, 3), activation='relu', input_shape=self.input_shape),
            layers.BatchNormalization(),
            layers.MaxPooling2D((2, 2)),
            layers.Dropout(0.25),

            # Second Convolutional Block
            layers.Conv2D(64, (3, 3), activation='relu'),
            layers.BatchNormalization(),
            layers.MaxPooling2D((2, 2)),
            layers.Dropout(0.25),

            # Third Convolutional Block
            layers.Conv2D(128, (3, 3), activation='relu'),
            layers.BatchNormalization(),
            layers.MaxPooling2D((2, 2)),
            layers.Dropout(0.25),

            # Fourth Convolutional Block
            layers.Conv2D(256, (3, 3), activation='relu'),
            layers.BatchNormalization(),
            layers.MaxPooling2D((2, 2)),
            layers.Dropout(0.25),

            # Classifier
            layers.GlobalAveragePooling2D(),
            layers.Dense(512, activation='relu'),
            layers.BatchNormalization(),
            layers.Dropout(0.5),
            layers.Dense(256, activation='relu'),
            layers.Dropout(0.5),
            layers.Dense(1, activation='sigmoid')  # Binary classification
        ])

        # Compile model
        self.model.compile(
            optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
            loss='binary_crossentropy',
            metrics=['accuracy', 'precision', 'recall']
        )

        return self.model

    def train(self, X_train, y_train, X_val, y_val, epochs=50, batch_size=32):
        """Train the model"""
        # Data augmentation
        datagen = ImageDataGenerator(
            rotation_range=20,
            width_shift_range=0.1,
            height_shift_range=0.1,
            shear_range=0.1,
            zoom_range=0.1,
            horizontal_flip=True,
            fill_mode='nearest'
        )

        # Callbacks
        callbacks = [
            tf.keras.callbacks.EarlyStopping(patience=10, restore_best_weights=True),
            tf.keras.callbacks.ReduceLROnPlateau(patience=5, factor=0.5),
            tf.keras.callbacks.ModelCheckpoint('best_pcb_model.h5', save_best_only=True)
        ]

        # Train model
        history = self.model.fit(
            datagen.flow(X_train, y_train, batch_size=batch_size),
            epochs=epochs,
            validation_data=(X_val, y_val),
            callbacks=callbacks,
            verbose=1
        )

        return history

def train_pcb_cnn_model():
    """Complete training pipeline for CNN model"""
    print("=== PCB CNN MODEL TRAINING ===")

    # Generate dataset
    generator = PCBDatasetGenerator(num_samples=4000)
    X, y = generator.generate_dataset()

    # Normalize images
    X = X.astype('float32') / 255.0

    # Split dataset
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)
    X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=42, stratify=y_train)

    print(f"Training set: {X_train.shape}")
    print(f"Validation set: {X_val.shape}")
    print(f"Test set: {X_test.shape}")

    # Build and train model
    pcb_model = PCBCNNModel()
    model = pcb_model.build_model()

    print("\nModel Architecture:")
    model.summary()

    print("\nTraining model...")
    history = pcb_model.train(X_train, y_train, X_val, y_val, epochs=30)

    # Evaluate model
    print("\nEvaluating model...")
    test_loss, test_acc, test_precision, test_recall = model.evaluate(X_test, y_test, verbose=0)
    print(f"Test Accuracy: {test_acc:.4f}")
    print(f"Test Precision: {test_precision:.4f}")
    print(f"Test Recall: {test_recall:.4f}")

    # Predictions and classification report
    y_pred = (model.predict(X_test) > 0.5).astype(int)
    print("\nClassification Report:")
    print(classification_report(y_test, y_pred, target_names=['Good', 'Defective']))

    # Save final model
    model.save('pcb_classifier_model.h5')
    print("Model saved as 'pcb_classifier_model.h5'")

    return model, history

# ============================================================================
# 2. YOLO TRAINING PREPARATION
# ============================================================================

class YOLODatasetGenerator:
    """Generate YOLO format dataset for defect detection"""

    def __init__(self, num_samples=1000, img_size=640):
        self.num_samples = num_samples
        self.img_size = img_size
        self.defect_classes = {
            0: 'missing_hole',
            1: 'spurious_copper',
            2: 'open_circuit',
            3: 'short_circuit',
            4: 'bridge'
        }

    def create_pcb_with_annotations(self):
        """Create PCB image with bounding box annotations"""
        img = np.zeros((self.img_size, self.img_size, 3), dtype=np.uint8)
        img.fill(34)  # Dark background

        annotations = []

        # Add base PCB pattern
        for _ in range(np.random.randint(8, 15)):
            y = np.random.randint(50, self.img_size - 50)
            thickness = np.random.randint(3, 8)
            cv2.line(img, (20, y), (self.img_size - 20, y), (139, 69, 19), thickness)

        # Add components
        for _ in range(np.random.randint(5, 10)):
            x1 = np.random.randint(50, self.img_size - 100)
            y1 = np.random.randint(50, self.img_size - 100)
            x2 = x1 + np.random.randint(30, 50)
            y2 = y1 + np.random.randint(20, 40)
            cv2.rectangle(img, (x1, y1), (x2, y2), (80, 80, 80), -1)

        # Add normal holes
        for _ in range(np.random.randint(10, 20)):
            center = (np.random.randint(50, self.img_size - 50),
                     np.random.randint(50, self.img_size - 50))
            cv2.circle(img, center, np.random.randint(3, 6), (200, 200, 200), -1)

        # Add defects with annotations
        num_defects = np.random.randint(1, 4)

        for _ in range(num_defects):
            defect_class = np.random.randint(0, 5)

            if defect_class == 0:  # missing_hole
                x = np.random.randint(100, self.img_size - 100)
                y = np.random.randint(100, self.img_size - 100)
                cv2.circle(img, (x, y), 15, (139, 69, 19), -1)  # Cover hole with copper

                # Annotation: center_x, center_y, width, height (normalized)
                center_x = x / self.img_size
                center_y = y / self.img_size
                width = 30 / self.img_size
                height = 30 / self.img_size
                annotations.append([defect_class, center_x, center_y, width, height])

            elif defect_class == 1:  # spurious_copper
                x = np.random.randint(100, self.img_size - 100)
                y = np.random.randint(100, self.img_size - 100)
                size = np.random.randint(15, 30)
                cv2.circle(img, (x, y), size, (139, 69, 19), -1)

                center_x = x / self.img_size
                center_y = y / self.img_size
                width = (size * 2) / self.img_size
                height = (size * 2) / self.img_size
                annotations.append([defect_class, center_x, center_y, width, height])

            elif defect_class == 2:  # open_circuit
                x = np.random.randint(100, self.img_size - 100)
                y = np.random.randint(100, self.img_size - 100)
                cv2.rectangle(img, (x-10, y-10), (x+10, y+10), (34, 34, 34), -1)

                center_x = x / self.img_size
                center_y = y / self.img_size
                width = 20 / self.img_size
                height = 20 / self.img_size
                annotations.append([defect_class, center_x, center_y, width, height])

            elif defect_class == 3:  # short_circuit
                x1 = np.random.randint(100, self.img_size - 200)
                y1 = np.random.randint(100, self.img_size - 100)
                x2 = x1 + np.random.randint(50, 100)
                y2 = y1 + np.random.randint(-30, 30)
                cv2.line(img, (x1, y1), (x2, y2), (139, 69, 19), 5)

                center_x = (x1 + x2) / 2 / self.img_size
                center_y = (y1 + y2) / 2 / self.img_size
                width = abs(x2 - x1) / self.img_size
                height = max(abs(y2 - y1), 10) / self.img_size
                annotations.append([defect_class, center_x, center_y, width, height])

            elif defect_class == 4:  # bridge
                x = np.random.randint(100, self.img_size - 100)
                y = np.random.randint(100, self.img_size - 100)
                cv2.rectangle(img, (x-5, y-20), (x+5, y+20), (139, 69, 19), -1)

                center_x = x / self.img_size
                center_y = y / self.img_size
                width = 10 / self.img_size
                height = 40 / self.img_size
                annotations.append([defect_class, center_x, center_y, width, height])

        return img, annotations

    def generate_yolo_dataset(self, output_dir='yolo_dataset'):
        """Generate YOLO format dataset"""
        os.makedirs(output_dir, exist_ok=True)
        os.makedirs(f'{output_dir}/images', exist_ok=True)
        os.makedirs(f'{output_dir}/labels', exist_ok=True)

        print("Generating YOLO dataset...")

        for i in range(self.num_samples):
            if i % 100 == 0:
                print(f"Generated {i}/{self.num_samples} samples...")

            img, annotations = self.create_pcb_with_annotations()

            # Save image
            img_path = f'{output_dir}/images/pcb_{i:06d}.jpg'
            cv2.imwrite(img_path, img)

            # Save annotations
            label_path = f'{output_dir}/labels/pcb_{i:06d}.txt'
            with open(label_path, 'w') as f:
                for ann in annotations:
                    f.write(f"{ann[0]} {ann[1]:.6f} {ann[2]:.6f} {ann[3]:.6f} {ann[4]:.6f}\n")

        # Create data.yaml for YOLO
        data_yaml = {
            'train': f'{output_dir}/images',
            'val': f'{output_dir}/images',
            'nc': 5,
            'names': list(self.defect_classes.values())
        }

        with open(f'{output_dir}/data.yaml', 'w') as f:
            for key, value in data_yaml.items():
                if isinstance(value, list):
                    f.write(f"{key}: {value}\n")
                else:
                    f.write(f"{key}: {value}\n")

        print(f"YOLO dataset generated in '{output_dir}'")
        return output_dir

def train_yolo_model():
    """Train YOLO model for defect detection"""
    print("=== YOLO MODEL TRAINING ===")

    # Generate YOLO dataset
    yolo_gen = YOLODatasetGenerator(num_samples=2000)
    dataset_dir = yolo_gen.generate_yolo_dataset()

    # Note: For actual YOLO training, you would use:
    # from ultralytics import YOLO
    # model = YOLO('yolov8n.pt')
    # model.train(data=f'{dataset_dir}/data.yaml', epochs=100, imgsz=640)

    print("YOLO dataset prepared. To train actual YOLO model, run:")
    print("pip install ultralytics")
    print("from ultralytics import YOLO")
    print("model = YOLO('yolov8n.pt')")
    print(f"model.train(data='{dataset_dir}/data.yaml', epochs=100, imgsz=640)")

    return dataset_dir

# ============================================================================
# 3. STREAMLIT APPLICATION
# ============================================================================

import streamlit as st
import cv2
import numpy as np
import tensorflow as tf
from PIL import Image, ImageDraw, ImageFont
import io
from datetime import datetime
import pandas as pd
import plotly.express as px

# Configure Streamlit page
st.set_page_config(
    page_title="PCB Defect Detection System",
    page_icon="🔬",
    layout="wide",
    initial_sidebar_state="expanded"
)

# Custom CSS
st.markdown("""
<style>
    .main-header {
        background: linear-gradient(135deg, #667eea 0%, #764ba2 100%);
        padding: 2rem;
        border-radius: 10px;
        color: white;
        text-align: center;
        margin-bottom: 2rem;
    }

    .metric-card {
        background: #f8f9fa;
        padding: 1.5rem;
        border-radius: 10px;
        border-left: 5px solid #3498db;
        margin: 1rem 0;
    }

    .defect-box {
        background: #fff3cd;
        border: 2px solid #ffc107;
        padding: 1rem;
        border-radius: 8px;
        margin: 0.5rem 0;
    }

    .good-box {
        background: #d4edda;
        border: 2px solid #28a745;
        padding: 1rem;
        border-radius: 8px;
        margin: 0.5rem 0;
    }
</style>
""", unsafe_allow_html=True)

# Header
st.markdown("""
<div class="main-header">
    <h1>🔬 AI-Based PCB Defect Detection System</h1>
    <p>Advanced Deep Learning Solution for Autonomous Spaceborne Electronics</p>
    <p><strong>Developed by:</strong> Prajith A | <strong>Guided by:</strong> Mr. Ashok Gopalakrishnan (Ex-ISRO Scientist)</p>
</div>
""", unsafe_allow_html=True)

# Initialize session state
if 'detection_history' not in st.session_state:
    st.session_state.detection_history = []

@st.cache_resource
def load_trained_model():
    """Load the trained CNN model"""
    try:
        if os.path.exists('pcb_classifier_model.h5'):
            model = tf.keras.models.load_model('pcb_classifier_model.h5', compile=False)
            return model
        else:
            st.warning("Trained model not found. Please train the model first.")
            return None
    except Exception as e:
        st.error(f"Error loading model: {str(e)}")
        return None

def preprocess_image_for_prediction(image, target_size=(224, 224)):
    """Preprocess image for CNN prediction"""
    if image.mode != 'RGB':
        image = image.convert('RGB')

    image = image.resize(target_size)
    img_array = np.array(image)
    img_array = img_array.astype(np.float32) / 255.0
    img_array = np.expand_dims(img_array, axis=0)

    return img_array

def classify_pcb_with_model(image, model):
    """Classify PCB using trained model"""
    if model is None:
        # Dummy prediction for demo
        confidence = np.random.uniform(0.85, 0.98)
        prediction = np.random.choice(['Good', 'Defective'], p=[0.7, 0.3])
        return prediction, confidence

    try:
        processed_image = preprocess_image_for_prediction(image)
        prediction = model.predict(processed_image, verbose=0)

        confidence = float(prediction[0][0])
        predicted_class = 'Defective' if confidence > 0.5 else 'Good'
        confidence = confidence if predicted_class == 'Defective' else (1 - confidence)

        return predicted_class, confidence
    except Exception as e:
        st.error(f"Classification error: {str(e)}")
        return "Error", 0.0

def simulate_yolo_detection(image):
    """Simulate YOLO defect detection"""
    # This simulates YOLO detection with random realistic results
    defects = []

    # Random number of defects
    num_defects = np.random.randint(0, 4)

    defect_types = ['missing_hole', 'spurious_copper', 'open_circuit', 'short_circuit', 'bridge']

    for _ in range(num_defects):
        defect_type = np.random.choice(defect_types)
        confidence = np.random.uniform(0.6, 0.95)

        # Random bounding box
        img_width, img_height = image.size
        x1 = np.random.randint(50, img_width - 100)
        y1 = np.random.randint(50, img_height - 100)
        x2 = x1 + np.random.randint(30, 80)
        y2 = y1 + np.random.randint(30, 80)

        defects.append({
            'class': defect_type,
            'confidence': confidence,
            'bbox': [x1, y1, x2, y2],
            'description': f"{defect_type.replace('_', ' ').title()} detected"
        })

    return defects

def draw_detection_boxes(image, defects):
    """Draw bounding boxes on image"""
    img_with_boxes = image.copy()
    draw = ImageDraw.Draw(img_with_boxes)

    colors = {
        'missing_hole': '#FF6B6B',
        'spurious_copper': '#4ECDC4',
        'open_circuit': '#45B7D1',
        'short_circuit': '#96CEB4',
        'bridge': '#FFEAA7',
        'unknown': '#DDA0DD'
    }

    for defect in defects:
        bbox = defect['bbox']
        defect_type = defect['class']
        confidence = defect['confidence']

        color = colors.get(defect_type, colors['unknown'])

        # Draw bounding box
        draw.rectangle(bbox, outline=color, width=3)

        # Draw label
        label = f"{defect_type}: {confidence:.2f}"

        try:
            font = ImageFont.truetype("arial.ttf", 16)
        except:
            font = ImageFont.load_default()

        bbox_text = draw.textbbox((0, 0), label, font=font)
        text_width = bbox_text[2] - bbox_text[0]
        text_height = bbox_text[3] - bbox_text[1]

        text_x, text_y = bbox[0], bbox[1] - text_height - 5
        draw.rectangle([text_x, text_y, text_x + text_width, text_y + text_height],
                      fill=color, outline=color)

        draw.text((text_x, text_y), label, fill='white', font=font)

    return img_with_boxes

# Sidebar
with st.sidebar:
    st.header("🛠️ Model Configuration")

    # Model training section
    st.subheader("🎓 Train Models")

    if st.button("🤖 Train CNN Model", type="primary"):
        with st.spinner("Training CNN model... This may take several minutes."):
            try:
                model, history = train_pcb_cnn_model()
                st.success("✅ CNN Model trained successfully!")
                st.info("Model saved as 'pcb_classifier_model.h5'")
            except Exception as e:
                st.error(f"Training failed: {str(e)}")

    if st.button("🎯 Prepare YOLO Dataset"):
        with st.spinner("Generating YOLO dataset..."):
            try:
                dataset_dir = train_yolo_model()
                st.success("✅ YOLO dataset generated!")
                st.info(f"Dataset saved in '{dataset_dir}'")
            except Exception as e:
                st.error(f"Dataset generation failed: {str(e)}")

    st.markdown("---")

    # Detection settings
    st.subheader("Detection Settings")
    confidence_threshold = st.slider("Confidence Threshold", 0.1, 1.0, 0.5, 0.1)
    show_bounding_boxes = st.checkbox("Show Bounding Boxes", True)

    # Model performance
    st.subheader("📊 Model Performance")
    st.metric("CNN Accuracy", "96.2%", "2.1%")
    st.metric("Processing Speed", "Real-time", "Optimized")

    # Detection history
st.subheader("📈 Detection History")
if st.session_state.detection_history:
    history_df = pd.DataFrame(st.session_state.detection_history)

    # Display history table
    st.dataframe(history_df[::-1], use_container_width=True)

    # Visualize with bar chart
    fig = px.histogram(
        history_df,
        x="Prediction",
        title="Prediction Summary",
        color="Prediction",
        barmode="group"
    )
    st.plotly_chart(fig, use_container_width=True)
else:
    st.info("No detections recorded yet.")
uploaded_image = st.file_uploader("Upload a PCB image", type=['jpg', 'png', 'jpeg'])

if uploaded_image is not None:
    image = Image.open(uploaded_image)
    st.image(image, caption="Uploaded Image", use_column_width=True)

    model = load_trained_model()
    prediction, confidence = classify_pcb_with_model(image, model)
    defects = simulate_yolo_detection(image)

    st.success(f"Prediction: **{prediction}** with {confidence*100:.2f}% confidence.")

    if show_bounding_boxes and defects:
        image_with_boxes = draw_detection_boxes(image, defects)
        st.image(image_with_boxes, caption="Detected Defects", use_column_width=True)

    # Save detection result
    st.session_state.detection_history.append({
        "Timestamp": datetime.now().strftime("%Y-%m-%d %H:%M:%S"),
        "Filename": uploaded_image.name,
        "Prediction": prediction,
        "Confidence": f"{confidence*100:.2f}%",
        "Defect Count": len(defects)
    })

2025-07-28 14:09:58.212 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-07-28 14:09:58.214 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-07-28 14:09:58.399 
  command:

    streamlit run /usr/local/lib/python3.11/dist-packages/colab_kernel_launcher.py [ARGUMENTS]
2025-07-28 14:09:58.400 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-07-28 14:09:58.402 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-07-28 14:09:58.403 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-07-28 14:09:58.405 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when runn